In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
miguelcorraljr_ted_ultimate_dataset_path = kagglehub.dataset_download('miguelcorraljr/ted-ultimate-dataset')
mitchellodili_carrierscsv_path = kagglehub.dataset_download('mitchellodili/carrierscsv')
gtun304_dataexpo1987_path = kagglehub.dataset_download('gtun304/dataexpo1987')

print('Data source import complete.')


In [ ]:
!pip install cudf dask-cudf \
    --extra-index-url=https://pypi.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
INFO: pip is looking at multiple versions of cudf to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement cubinlinker (from cudf) (from versions: none)
ERROR: No matching distribution found for cubinlinker


In [ ]:
!pip install --upgrade dask distributed


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.4 MB/s eta 0:00:00
  Attempting uninstall: dask
    Found existing installation: dask 2024.11.2
    Uninstalling dask-2024.11.2:
      Successfully uninstalled dask-2024.11.2
  Attempting uninstall: distributed
    Found existing installation: distributed 2024.9.0
    Uninstalling distributed-2024.9.0:
      Successfully uninstalled distributed-2024.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
dask-expr 1.1.19 requires dask==2024.11.2, but you have dask 2024.12.0 which is incompatible.
rapids-dask-dependency 24.10.0a0 requires dask==2024.9.0, but you have dask 2024.12.0 which is incompatible.
rapids-dask-dependency 24.10.0a0

In [ ]:
import cudf
import dask_cudf
import pandas as pd
import dask.dataframe as dd
import time
import os
from dask_cuda import LocalCUDACluster
from dask.distributed import Client, wait

try:
    client.close()
    cluster.close()
except:
    pass

cluster = LocalCUDACluster(dashboard_address=":8791", n_workers=2, threads_per_worker=1, memory_limit='20GB')
client = Client(cluster)
print(client)

!nvidia-smi || echo "nvidia-smi falhou"

<Client: 'tcp://127.0.0.1:43799' processes=2 threads=2, memory=37.25 GiB>
Thu Dec 12 02:54:56 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                       

In [ ]:
flights_file = "/kaggle/input/dataexpo1987/1987.csv"
carriers_file = "/kaggle/input/carrierscsv/carriers.csv"
esquema = {
    "Year": "int64",
    "Month": "int64",
    "DayofMonth": "int64",
    "DayOfWeek": "int64",
    "DepTime": "float64",
    "CRSDepTime": "float64",
    "ArrTime": "float64",
    "CRSArrTime": "float64",
    "UniqueCarrier": "str",
    "FlightNum": "int64",
    "TailNum": "str",
    "ActualElapsedTime": "float64",
    "CRSElapsedTime": "float64",
    "AirTime": "float64",
    "ArrDelay": "float64",
    "DepDelay": "float64",
    "Origin": "str",
    "Dest": "str",
    "Distance": "float64",
    "TaxiIn": "float64",
    "TaxiOut": "float64",
    "Cancelled": "bool",
    "CancellationCode": "str",
    "Diverted": "bool",
    "CarrierDelay": "float64",
    "WeatherDelay": "float64",
    "NASDelay": "float64",
    "SecurityDelay": "float64",
    "LateAircraftDelay": "float64"
}



In [ ]:
flights_dask = dd.read_csv(flights_file, dtype=esquema, blocksize=5e6)
flights_cudf = cudf.read_csv(flights_file)
carriers = cudf.read_csv(carriers_file)

def measure_time(func, *args):
    start_time = time.time()
    result = func(*args)
    end_time = time.time()
    return result, end_time - start_time

def max_arr_delay_cudf(df):
    return df[df["Cancelled"] == False].groupby("UniqueCarrier")["ArrDelay"].max().nlargest(1).index[0]

def most_flights_day_cudf(df):
    return df[df["Cancelled"] == False].groupby("DayOfWeek").size().nlargest(1).index[0]

def most_cancellations_cudf(df):
    return df[df["Cancelled"] == True].groupby("UniqueCarrier").size().nlargest(1).index[0]

def most_miles_cudf(df):
    return df[df["Cancelled"] == False].groupby("UniqueCarrier")["Distance"].sum().nlargest(1).index[0]

def most_recovery_cudf(df):
    df["Recovery"] = df["DepDelay"] - df["ArrDelay"]
    return df[df["Cancelled"] == False].groupby("UniqueCarrier")["Recovery"].sum().nlargest(1).index[0]

def growth_flights_cudf(df):
    return df[df["Cancelled"] == False].groupby("Year").size()

def cancellations_by_month_cudf(df):
    return df[df["Cancelled"] == True].groupby("Month").size()

def cancel_percent_cudf(df):
    flights_by_month = df.groupby("Month").size()
    cancellations = df[df["Cancelled"] == True].groupby("Month").size()
    return (cancellations / flights_by_month) * 100

def max_arr_delay_pd(df):
    return df[df["Cancelled"] == False].groupby("UniqueCarrier")["ArrDelay"].max().idxmax()

def most_flights_day_pd(df):
    return df[df["Cancelled"] == False].groupby("DayOfWeek").size().idxmax()

def most_cancellations_pd(df):
    return df[df["Cancelled"] == True].groupby("UniqueCarrier").size().idxmax()

def most_miles_pd(df):
    return df[df["Cancelled"] == False].groupby("UniqueCarrier")["Distance"].sum().idxmax()

def most_recovery_pd(df):
    df["Recovery"] = df["DepDelay"] - df["ArrDelay"]
    return df[df["Cancelled"] == False].groupby("UniqueCarrier")["Recovery"].sum().idxmax()

def growth_flights_pd(df):
    return df[df["Cancelled"] == False].groupby("Year").size()

def cancellations_by_month_pd(df):
    return df[df["Cancelled"] == True].groupby("Month").size()

def cancel_percent_pd(df):
    flights_by_month = df.groupby("Month").size()
    cancellations = df[df["Cancelled"] == True].groupby("Month").size()
    return (cancellations / flights_by_month) * 100

def compare_execution_times():
    try:
        flights_pd = flights_dask.compute()
        print("Operação compute foi bem-sucedida.")
    except Exception as e:
        print(f"Erro na operação compute: {e}")
        return

    functions_cudf = [
        ("Maior atraso de chegada", max_arr_delay_cudf),
        ("Dia com mais vôos", most_flights_day_cudf),
        ("Maior número de cancelamentos", most_cancellations_cudf),
        ("Mais milhas voadas", most_miles_cudf),
        ("Melhor recuperação de atrasos", most_recovery_cudf),
        ("Crescimento no número de vôos", growth_flights_cudf),
        ("Cancelamentos por mês", cancellations_by_month_cudf),
        ("Cancelamentos em relação aos vôos mensais", cancel_percent_cudf)
    ]

    functions_pd = [
        ("Maior atraso de chegada", max_arr_delay_pd),
        ("Dia com mais vôos", most_flights_day_pd),
        ("Maior número de cancelamentos", most_cancellations_pd),
        ("Mais milhas voadas", most_miles_pd),
        ("Melhor recuperação de atrasos", most_recovery_pd),
        ("Crescimento no número de vôos", growth_flights_pd),
        ("Cancelamentos por mês", cancellations_by_month_pd),
        ("Cancelamentos em relação aos vôos mensais", cancel_percent_pd)
    ]

    print("Comparação de tempos de execução entre cudf e pandas:")
    for ((desc_cudf, func_cudf), (desc_pd, func_pd)) in zip(functions_cudf, functions_pd):
        result_cudf, time_cudf = measure_time(func_cudf, flights_cudf)
        result_pd, time_pd = measure_time(func_pd, flights_pd)
        print(f"{desc_cudf}:")
        print(f"  cudf: {result_cudf} (Tempo de execução: {time_cudf:.4f} segundos)")
        print(f"  pandas: {result_pd} (Tempo de execução: {time_pd:.4f} segundos)")



In [ ]:
try:
    client.get_versions(check=True)
    print("Conexão com o scheduler bem-sucedida.")

except Exception as e:
    print(f"Erro ao conectar ao scheduler: {e}")

compare_execution_times()


Conexão com o scheduler bem-sucedida.
Operação compute foi bem-sucedida.
Comparação de tempos de execução entre cudf e pandas:
Maior atraso de chegada:
  cudf: PS (Tempo de execução: 0.2498 segundos)
  pandas: PS (Tempo de execução: 0.3444 segundos)
Dia com mais vôos:
  cudf: 4 (Tempo de execução: 0.0627 segundos)
  pandas: 4 (Tempo de execução: 0.2619 segundos)
Maior número de cancelamentos:
  cudf: UA (Tempo de execução: 0.0180 segundos)
  pandas: UA (Tempo de execução: 0.0160 segundos)
Mais milhas voadas:
  cudf: AA (Tempo de execução: 0.0375 segundos)
  pandas: AA (Tempo de execução: 0.3226 segundos)
Melhor recuperação de atrasos:
  cudf: CO (Tempo de execução: 0.0526 segundos)
  pandas: CO (Tempo de execução: 0.3279 segundos)
Crescimento no número de vôos:
  cudf: Year
1987    1292141
dtype: int64 (Tempo de execução: 0.0389 segundos)
  pandas: Year
1987    1292141
dtype: int64 (Tempo de execução: 0.2809 segundos)
Cancelamentos por mês:
  cudf: Month
11     5191
10     3001
12    1

A comparação entre cuDF e Pandas evidencia que o cuDF oferece vantagens significativas.. No entanto, a configuração do cuDF foi bastante desafiadora, gerando inúmeros erros ao longo do processo, desde incompatibilidades de versões até ajustes específicos no código. Apesar do esforço adicional para sua implementação, os ganhos em escalabilidade e tempo de execução demonstram que a ferramenta é eficiente em cenários de grande volume de dados, embora exija mais conhecimento técnico e tempo para configuração inicial.


- **Maior atraso de chegada:** cuDF foi cerca de 38% mais rápido (0.2498s contra 0.3444s).
- **Dia com mais voos:** cuDF apresentou uma vantagem significativa, sendo aproximadamente 76% mais rápido (0.0627s contra 0.2619s).
- **Maior número de cancelamentos:** Pandas foi ligeiramente mais rápido, mas a diferença foi insignificante (0.0180s contra 0.0160s).
- **Mais milhas voadas:** cuDF teve um desempenho substancialmente superior, com uma redução de ~88% no tempo (0.0375s contra 0.3226s).
- **Melhor recuperação de atrasos:** cuDF foi cerca de 84% mais rápido (0.0526s contra 0.3279s).
- **Crescimento no número de voos:** cuDF foi consideravelmente mais eficiente, com ~86% de redução no tempo (0.0389s contra 0.2809s).
- **Cancelamentos por mês:** cuDF foi levemente mais rápido, com uma redução de ~26% no tempo (0.0120s contra 0.0162s).
- **Cancelamentos/vôos mensais:** Pandas foi mais eficiente nesta tarefa, sendo cerca de 57% mais rápido (0.1068s contra 0.0459s).
